## Chapter 1: "An introduction to PyTorch"

From now on, we will be working with a Python package called PyTorch. Besides TensorFlow, it is the premier machine learning environment. It comes with a lot of features and a couple things that may trip up new users. We will try to slowly ease ourselves into using PyTorch.

For all future tasks, try to figure out what it is you want to do in the jargon of Python/PyTorch and find the function(s) you need yourself. This may be slow at first, but it helps you get the hang of solving problems on your own. Try to say what you want as precisely as possible. Then google exactly that, in as few words as possible, and try to understand what you found and how it works. If you don't understand the solution you found, take it apart, execute only a part of it, experiment!

If something still doesn't work, or the assignment doesn't make sense, feel free to ask the tutors, too. After all, that's why we're here.

### Chapter 1.1: "Tensors"

The tensor sits at the heart of the PyTorch package. A tensor in machine learning is basically the same as an n-dimensional matrix. Under the hood, tensors work almost the same way as the numpy arrays we already know. However, tensors come with a couple of additional functionalities. Attached to each tensor is information about the data type it contains, the device it currently resides in (for example the regular memory, or the memory of a GPU), and most importantly information about its gradients. We will come back to the last bit later on.

In [ ]:
import torch
import numpy as np

# How do we make a tensor? Essentially the same as a numpy array.
my_tensor = torch.tensor([1, 2, 3])
print(my_tensor)

In [ ]:
# In fact, we can even make a numpy array into a tensor:
my_array = np.array([2, 3, 4, 5])
my_tensor = torch.tensor(my_array)
print(my_tensor)

In [ ]:
# We can also get back our numpy array from a tensor:
my_array = my_tensor.numpy()
print(my_array)

In [ ]:
# Tensors come with most of the same functionality as numpy arrays, such as
# Knowing its own dimensions
my_tensor = torch.zeros((1, 4))
print(my_tensor.size())

In [ ]:
# Transposition
print(my_tensor.T)

In [ ]:
# Reshaping
print(my_tensor.reshape((2, 2)))

In [ ]:
# Flattening
print(my_tensor.flatten())

In [ ]:
# Operations along an axis
my_tensor = torch.tensor([[1, 2], [4, 3]])
print(my_tensor.argmax(axis=0))
print(my_tensor.argmax(axis=1))
# and so on.

In [ ]:
# Occasionally, you will come across tensors with only one component.
# These are still, technically, a sort of matrix. However, you can turn
# them into just a regular scalar if need be:
t = torch.tensor(1)
print(t)
print(t.item())

# As you may have already noticed, the syntax of the tensor operations is
# almost the same as that for numpy arrays, but not always exactly the same.
# Don't worry - PyTorch is well-documented. If you know what you want,
# you can find it on Google or their documentation website.

In [ ]:
# Tensors know where they currently live.
# This attribute of a tensor is called 'device'.
my_tensor = torch.tensor([[1, 2], [3, 4]])
print(my_tensor.device)

# We can manually move tensors and other PyTorch objects around on devices.
# We do so with the '.to' method. The method expects the name of the device
# we send it to. This could be "cpu", "cuda", or something like "cuda:0".

# 'cuda' is the name of the engine under the hood, so to speak, and when we
# specify 'cuda' as the target device, our tensor is put into the memory of
# a GPU. If we specify the number, we select the nth GPU specifically.
my_tensor = my_tensor.to("cuda:0")
print(my_tensor.device)

### Chapter 1.2: "Image data"

We will be slinging some significant numbers of images around later, enough that we do not want to write the name of every image manually or look at all of them. Therefore, let's look at how we can start with an image on our disk and end up with a PyTorch tensor containing said image in our memory.

For this purpose we have prepared some tutorial images.

In [ ]:
import PIL
import matplotlib.pyplot as plt
import numpy as np
import torch, torchvision

# Let's first load an image. For this purpose, we use PIL ('Pillow'), 
# which can read jpg or png images from the disk.
with PIL.Image.open("../data/Clean_LiTS/train/volumes/volume-10_337.png") as f:

    # Now we grab the array from the image.
    img_array = np.array(f, dtype = np.uint8)

# Let's see what size our image is.
print(img_array.shape)

In [ ]:
# We can already visualize the image using matplotlib
plt.figure()
plt.imshow(img_array)
plt.xlim((0, 256))
plt.ylim((0, 256))

During this course, we will primarily be working with a dataset named "LiTS". LiTS is shorthand for "**Li**ver **T**umor **S**egmentation". It contains computed tomography (CT) images, which are 3D volumes (X, Y, Z) reconstructed from measured x-ray absorption. These volumes consist of several hundred *slices* each. A slice is nothing more than a 2-dimensional (X, Y) grayscale image. In fact, the image above is one of the LiTS images. The goal in the LiTS challenge was to make a neural network accurately locate the liver in those images, as well as the tumors, if there were any.

Later on, we will first try to classify the images into three categories: No Liver, Liver, and Liver Tumor. We will later also try to solve the segmentation challenge with our neural networks. 

For now, let's just figure out how to work with the data.

In [ ]:
# Lets make our plot look a little nicer.

# Let us first load the image
with PIL.Image.open("../data/Clean_LiTS/train/volumes/volume-10_337.png") as f:
    
    # This time, we convert the image to grayscale immediately.
    # It always was already grayscale of course, but had 3 channels - RGB.
    f = f.convert("L")

    # We again grab the array from the image.
    img_array = np.array(f, dtype = np.uint8)

# This time, we also load the segmentation of the liver and potential tumors.
with PIL.Image.open("../data/Clean_LiTS/train/segmentations/segmentation-10_livermask_337.png") as f:
    f = f.convert("L")
    liver = np.array(f, dtype = np.uint8)
with PIL.Image.open("../data/Clean_LiTS/train/segmentations/segmentation-10_lesionmask_337.png") as f:
    f = f.convert("L")
    tumors = np.array(f, dtype = np.uint8)

# We make what is called a meshgrid - they are a tool designed for exactly the
# kind of plot that we are making right now. A meshgrid is essentially every
# combination of x and y coordinates in our image.
X, Y = np.meshgrid(np.arange(256), np.arange(256))
plt.figure()
# We show the image again, but since we only have one channel now,
# we get to use colormaps, which look a lot nicer than regular grayscale images.
plt.imshow(img_array, cmap = "bone")
plt.xlim((0, 256))
plt.ylim((0, 256))
# We can draw the 'truth' into our image with the help of the countour function
plt.contour(X, Y, liver, colors = "g", alpha = 0.25, linewidths = 0.5)
plt.contour(X, Y, tumors, colors = "r", alpha = 0.25, linewidths = 0.5)

Now that we know what the image looks like, let's figure out how to give it to our GPU.

PyTorch expects a very specific order of dimensions for its inputs: $B * C * H * W$, where $B$ is the number of images in a batch (images I work with in parallel), C is the number of channels it has (such as RGB), and H and W are height and width. opencv (the library that PIL is built on) orders them the other way around: $H * W * C$. Helpfully, PyTorch knows about PIL, and we can turn the PIL Image that we have seen before directly into a PyTorch tensor using a function called pil_to_tensor. As you can see, it also puts the dimensions in the right order!

In [ ]:
# Read in the image as a PIL Image
image = PIL.Image.open('../data/Clean_LiTS/train/volumes/volume-10_337.png')

# Convert the PIL image to Torch tensor
img_tensor = torchvision.transforms.functional.pil_to_tensor(image)

# Print the shape of the original image, and the converted tensor
print(np.array(image).shape)
print(img_tensor.size())

In [ ]:
# All that is left now, is to push the image to the GPU, if I am so inclined:

img_tensor = img_tensor.to("cuda")

### Chapter 1.3: "The recipe"

Now that we have an understanding of what kind of image we are loading, and how to load them properly, let's get to the core of the machine learning with neural networks. Below, you will see the basic recipe for training a neural network.

In practice, only very few steps are necessary to train a neural network. The more sophisticated your training algorithm and the more complex your data, the more you will eventually move away from this recipe. Conceptually, however, the steps remain largely the same:

- I) Take the input images and do some preprocessing to them. This includes things such as guaranteeing the images to all have the same shape, cutting off uninteresting parts of the image, or performing image augmentations like adding noise.
- II) Feed them to the GPU.
- III) Let the model make some predictions.
- IV) Compute the loss function from the predictions and the ground truths. ("How wrong were my predictions?")
- V) Derive the gradients of the model parameters with respect to the loss. ("In which direction must my parameters move to decrease my loss and thereby improve earlier predictions?")
- VI) Move all model parameters into this direction for a certain distance. ("Ask, how far must I go in this direction, then go.")
- VII) Repeat 1-6 until the predictions become good.
- VIII) Occasionally validate your performance on previously unseen data to check whether they are good.
- IX) Finally, evaluate the model on previously unseen data. This is the final result, and the thing reported in papers.

For now, we will go through the recipe and get an understanding of what we do and why. Once we understand this, we can start replacing the ready-to-go code provided by PyTorch with our own code. If something does not work later on, it can always be good to go back to the recipe and try to see what the recipe does that you maybe don't.

In [ ]:
# This example will actually train something, although it is not particularly sophisticated.
# Try to execute it, and try to follow what each part of the code does!

# 1) Imports
"""
First, we handle our imports. Typically we do this at the top of our
program. If looking at the text starts bothering you, you are free to
delete it, edit it, or click the little arrow next to the quotation
marks to hide the text in a single line.
"""

import torch, torchvision
import numpy as np
import matplotlib.pyplot as plt
import PIL
import os

# We import some helper functions that we made, too. Later, you will
# make these functions yourself.
import sys
sys.path.append("/Volumes/PortableSSD/MLCourse/Course_Materials")
import utility.utils as uu

# We also check if CUDA (read: at least one GPU) is available
device = ("cuda" if torch.cuda.is_available() else "cpu")

# 2) The Dataset
"""
PyTorch has specific requirements that datasets need to fulfill.
A dataset is a class and it handles your data (surprise). In the
next chapter, we will make our own, and we will discuss in more
detail what actually needs to be done to replace one.

For now, we will stick with a pre-existing one, called CIFAR10.
CIFAR10 contains images of flowers, which we want to try and classify.

(Since someone thought it would be a fantastic idea for the dataset to
return PIL Images instead of tensors, we add a ToTensor transform. We
will come back to transforms and how to use them later. For now, you
don't have to concern yourself with that.)

Finally, since we will want to gain an understanding of the actual
performance we have reached, we will split our dataset into a train,
validation, and test set, containing 80%, 10% and 10% of the data
respectively.
"""
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
dataset = torchvision.datasets.CIFAR10("../data/CIFAR10/", download = True, transform = transforms)
dl = len(dataset)
train_set, val_set, test_set = torch.utils.data.random_split(dataset = dataset, lengths = [int(0.8*dl), int(0.1*dl), int(0.1*dl)])

# 3) The Dataloader
"""
A dataloader wraps a dataset. It does not do all that much.
For the most part, it just grabs random data points from your dataset,
glues the tensors together along the first dimension and returns them.
We will discuss in more detail the options that the dataloader affords
us, but we will not have to write our own.

Let's quickly walk through what we are telling the dataloader:
dataset - This is the dataset from the step before. We make one dataloader
            for each of the subsets we have created.
batch_size - How many images do we want our dataloader to give us at
            once.
num_workers - The dataloader outsources the loading of the data to a
            number of processes to speed things up. We can select the
            number of processes here. Specifying "0" means that the
            main process will do all the loading. This is a bit slower
            but sometimes more useful for debugging.
shuffle - Whether we want to shuffle our dataset before loading. As a
            general rule, you should always shuffle your dataset during
            training so the neural network sees different combinations
            of data.
drop_last - Imagine you have 100 images. Your batch size is 32. You can
            create 3 proper batches, which have a length of 32 along the
            first dimension. Your final batch, before you start the next
            epoch and go through your dataset again, will contain only 4
            images. A lot of the time, we do not want such an incomplete
            batch! Why this is the case, we will learn a little later.
"""
batch_size = 128
train_loader = torch.utils.data.DataLoader(
    dataset = train_set,
    batch_size = batch_size, 
    num_workers = 8, 
    shuffle = True, 
    drop_last = True
    )

val_loader = torch.utils.data.DataLoader(
    dataset = val_set,
    batch_size = batch_size, 
    num_workers = 0, 
    shuffle = False, 
    drop_last = False
    )

test_loader = torch.utils.data.DataLoader(
    dataset = test_set,
    batch_size = batch_size, 
    num_workers = 0, 
    shuffle = False, 
    drop_last = False
    )

# 4) The Model
"""
If machine learning is a meal, then the model is the meat.
The model contorts and compresses the input images into a low-dimensional
space, according to its architecture and current weights. It makes some
prediction for the input images, which will later be evaluated. The model
is later corrected according to this evaluation. Most of the work later
during the course will be to create our own models, and recreate the most
successful and inflential ones from the history of the field.

For the moment, we will simply pick up a pre-existing model and not make
one ourselves. The only thing we change is the final layer of the model,
since the model was originally designed for different data. Don't concern
yourself with that last step for now, we will come back to this.
"""
model = torchvision.models.resnet18(pretrained = True)
model.fc = torch.nn.Linear(model.fc.in_features, 10)
model = model.to(device)

# 5) The Loss function
"""
After the model has made a prediction, we want to know, how good it was.
In order for the model to be able to benefit from this representation of
its quality, said function needs to fulfill several criteria: It has to
be differentiable (almost) everywhere, so we can derive gradients for
our model parameters from it and it needs to return lower values, the
better the predictions and targets match up.

Mathematically, we compute the loss function, a measure of how bad our
predictions for the current data were. Later, we optimize our network
parameters by computing the gradient of the loss function with respect
to each specific parameter. This step is fully automated in PyTorch.

For any set of modules which have a forward function that performed
differentiable operations, the '.backward()' function will be created
by PyTorch - we do not have to calculate anything manually.

Writing such a loss function is comparatively easy, and we will be doing
so much later during the course. For now, we will use an out-of-the-box
loss function that PyTorch provides, called CrossEntropyLoss. You can
find the exact formula of how predictions are "rated" here:
https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
"""
loss_criterion = torch.nn.CrossEntropyLoss()

# 6) The Optimizer
"""
This is where a large part of the magic happens. The optimizer has access
to all of a model's parameters. When each parameter has figured out in
which 'direction' the greatest improvement to the earlier predictions was
to be found, the optimizer figures out, how far to actually move the
parameters in this direction.

(Good) optimizers are quite complicated to write, and few people fully
understand the complex math involved in guaranteeing convergence and
stability of the learning process. We may look at a couple different ones
and give some overview of how they operate. However, we will not be making
our own.
"""
optimizer = torch.optim.Adam(params = model.parameters(), lr = 1e-4, weight_decay = 1e-2)

# 7) The Training Loop
"""
Now it's time to stick all of it together.
We follow steps I-IX from the start of the chapter.
"""

num_epochs = 10

for epoch in range(num_epochs):

    # I) Grab my data
    for step, (data, targets) in enumerate(train_loader):

        # This command is a peculiarity of PyTorch.
        # PyTorch accumulates gradients of parameters instead of overwriting
        # them when new predictions are made. This is a useful feature, but
        # most of the time, people do not use it. zero_grad() manually resets
        # this accumulation process.
        optimizer.zero_grad()

        # II) Put it onto the GPU.
        data, targets = data.to(device), targets.to(device)

        # III) Make some predictions.
        predictions = model(data)

        # IV) How wrong were our predictions?
        loss = loss_criterion(predictions, targets)
        
        # At this point it's also useful to print out performance indicators.
        if step % 50 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}]\t Step [{step+1}/{len(train_set)//batch_size}]\t Loss: {loss.item():.4f}")

        # V) In which direction do we have to go to make them better?
        loss.backward()

        # VI) How far do I have to go in this direction?
        # Once I know it, move the parameters accordingly.
        optimizer.step()

        # VII) Repeat.
    
    # VIII) Let's occasionally validate our model's performance on unseen data.
    if epoch % 5 == 0 and not epoch == 0:
    
        # We first tell our model to not try and train parameters for now.
        model.eval()

        # We also tell PyTorch to not collect any gradients for the time being.
        with torch.no_grad():

            hits = 0
            losses = []
            batch_sizes = []

            # This time, we grab the validation data our model has not seen before.
            for step, (data, targets) in enumerate(val_loader):

                data, targets = data.to(device), targets.to(device)
                predictions = model(data)
                loss = loss_criterion(predictions, targets)
                losses.append(loss.item())
                batch_sizes.append(data.size()[0])

                # In the hope that our performance during validation is indicative
                # of our performance on the test set (which is usually reported in
                # papers and challenges), let's measure our actual accuracy.
                # First, we convert our predictions from a tensor of dimensions
                # N x C (batch size x classes) to one of dimension N.
                class_predictions = torch.argmax(predictions, dim = 1).flatten()
                # Then, we count the number of correct predictions.
                hits = hits + sum([1 if cp == t else 0 for cp, t in zip(class_predictions, targets)])

            accuracy = hits / len(val_set)
            avg_loss = sum([l * bs for l, bs in zip(losses, batch_sizes)]) / sum(batch_sizes)
            print(f"Epoch: {epoch},\t Validation Loss: {avg_loss:.4f},\t Accuracy: {accuracy:.4f}")

        # After we are done validating, let's not forget to go back to storing gradients!
        model.train()

# IX) Let's check our final performance.

Overwhelmed? That is very understandable. Not overwhelmed? Even better!

For now, before we break everything apart and look at the innards, let's first try to get a little bit of a handle on things. See if you can answer the following questions, or, more to the point, if you can edit the code in question successfully.

1. Try to complete step IX) on your own. It should look almost the same as the validation.
2. Try to replace the dataset (for example with CIFAR100) and see what happens. Edit the code so that CIFAR100 trains successfully for a little while.
3. Try to replace the model with another pre-existing model (maybe another ResNet, or maybe another model entirely) and see what happens. Do you see where it breaks down? Try to adapt the code to successfully use another model.
4. Try running the model on the CPU instead of the GPU.

You can edit the recipe above, or copy it over to below and start editing there. You are welcome to throw out any and all parts if you want to test something. *If there is any questions at this point, always ask - no questions are stupid!*

In [ ]:
# Solutions

# == Task 1 ==
# The solution here can simply be appended at the end of the code that already exists.

# First, disable model updates again.
model.eval()

# Disable gradient collecting for the moment.
with torch.no_grad():

    hits = 0
    losses = []
    batch_sizes = []

    # This time, we grab the test data our model has not seen before.
    for step, (data, targets) in enumerate(test_loader):

        data, targets = data.to(device), targets.to(device)
        predictions = model(data)
        loss = loss_criterion(predictions, targets)
        losses.append(loss.item())
        batch_sizes.append(data.size()[0])
        class_predictions = torch.argmax(predictions, dim=1).flatten()
        hits = hits + sum([1 if cp == t else 0 for cp, t in zip(class_predictions, targets)])

    accuracy = hits / len(test_set)
    avg_loss = sum([l * bs for l, bs in zip(losses, batch_sizes)]) / sum(batch_sizes)
    print(f"Final testing - Test Loss: {avg_loss:.4f},\t Accuracy: {accuracy:.4f}")

# == Task 2 ==
# All that needs to be done is to 
# 1) replace the dataset's name with CIFAR100,
# 2) and change the number of output nodes in model.fc to 100.

# == Task 3 ==
# Any other ResNet should work just fine. Other models should usually not.
# The reason is that the pre-existing models all use their own names for
# the different layers. Hence, we need to find out the name of the final
# layer.

# == Task 4 ==
# Exchange the line
#device = ("cuda" if torch.cuda.is_available() else "cpu")
# with
#device = "cpu"

In [ ]:
# We can save our model to the disk like so ('../models' means 'go back 1 directory, then into 'models'):
def save_model(name, model):
    loc = os.path.join("../models", name)
    os.makedirs("../models", exist_ok = True)

    # To save a DataParallel model generically, save the model.module.state_dict().
    # This way, you have the flexibility to load the model any way you want to any device you want.
    if isinstance(model, torch.nn.DataParallel):
        torch.save(model.module.state_dict(), loc)
    else:
        torch.save(model.state_dict(), loc)

save_model("MLC_1.tar", model)